In [1]:
!pip install geopandas ipyleaflet psycopg2

In [2]:
import ipyleaflet as ipy 
import ipywidgets as ipyw
import geopandas 
import json
import psycopg2 
import getpass


In [3]:
DGGS_COLUMN_LOOKUP = {
       "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/": "sa1_main16",
       "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/": "sa2_main16",
       "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel3/": "sa3_code16",
        # "asgs16_sa4": "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel4",
        # "asgs16_mb": "http://linked.data.gov.au/dataset/asgs2016/meshblock",
        "http://linked.data.gov.au/dataset/asgs2016/stateorterritory/": "sta_name",  # need further mapting from code to name
        # "asgs16_sua": "http://linked.data.gov.au/dataset/asgs2016/significanturbanarea",
        # "asgs16_ireg": "http://linked.data.gov.au/dataset/asgs2016/indigenousregion",
        # "asgs16_iloc": "http://linked.data.gov.au/dataset/asgs2016/indigenouslocation",
        # "asgs16_iare": "http://linked.data.gov.au/dataset/asgs2016/indigenousarea",
        # "asgs16_ra": "http://linked.data.gov.au/dataset/asgs2016/remotenessarea",
        # "asgs16_gccsa": "http://linked.data.gov.au/dataset/asgs2016/greatercapitalcitystatisticalarea",
        # "asgs16_ucl": "http://linked.data.gov.au/dataset/asgs2016/urbancentreandlocality",
        # "asgs16_sosr": "http://linked.data.gov.au/dataset/asgs2016/sectionofstaterange",
        # "asgs16_sos": "http://linked.data.gov.au/dataset/asgs2016/sectionofstate",
        "http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/": "lga_code19",
        # "asgs16_ced": "http://linked.data.gov.au/dataset/asgs2016/commonwealthelectoraldivision": "ced_state", # need further mapping from code to name
        "http://linked.data.gov.au/dataset/asgs2016/statesuburb/": "ssc_code16",
        # "asgs16_nrmr": "http://linked.data.gov.au/dataset/asgs2016/naturalresourcemanagementregion",
        # "geofabric2_1_1_ahgfcontractedcatchment": "http://linked.data.gov.au/dataset/geofabric/contractedcatchment",
        # "geofabric2_1_1_riverregion": "http://linked.data.gov.au/dataset/geofabric/riverregion",
        # "geofabric2_1_1_awradrainagedivision": "http://linked.data.gov.au/dataset/geofabric/drainagedivision"
    }
STATE_TO_CODE = {
    "1": "New South Wales",
    "2": "Victoria",
    "3": "Queensland",
    "4": "South Australia", 
    "5": "Western Australia",
    "6": "Tasmania",
    "7": "Northern Territory",
    "8": "Australian Capital", 
    "9": ''
}

In [4]:
PG_HOST = 'location-index.cx6rhroaeuxv.ap-southeast-2.rds.amazonaws.com'
PG_PORT = '5432'
PG_DB_NAME = 'postgres'
PG_USER = 'govreadonly'
PG_PASSWORD = getpass.getpass(prompt='DGGS Database Password:: ', stream=None) 
PG_ENDPOINT = f'host={PG_HOST} port={PG_PORT} dbname={PG_DB_NAME} user={PG_USER} password={PG_PASSWORD}'

DGGS Database Password:: ········


In [5]:
def find_dggs_by_loci_uri(uri):
    """
    Function for finding an array of DGGS cells by a loci uri, eg: http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/31503140814
    """
    dggs_column = 'sa1_main16'
    uri_value = ""
    for lookup_key, lookup_value in DGGS_COLUMN_LOOKUP.items():
        index = uri.find(lookup_key)
        if index==0:
            dggs_column = lookup_value
            uri_value = uri[len(lookup_key):len(uri)]
            break
    sql = f'select auspix_dggs from public."MainTable01" where {dggs_column}=\'{uri_value}\''
#     print(sql)
    conn = psycopg2.connect(PG_ENDPOINT)
    db_cursor = conn.cursor()
    res = db_cursor.execute(sql)
    records = db_cursor.fetchall() 
    db_cursor.close()
    result = []
    for record in records:
        result.append(record)
    return result

In [6]:
%%time
dggs_cells = find_dggs_by_loci_uri('http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/60302107105')


UndefinedTable: relation "public.MainTable01" does not exist
LINE 1: select auspix_dggs from public."MainTable01" where sa1_main1...
                                ^


In [8]:
def gen_dggs_array(db_return_dggs):
    return [dggs[0] for dggs in db_return_dggs]
print(gen_dggs_array(dggs_cells))

['S3006888510', 'S3006888832', 'S3006888833', 'S3006888834', 'S3006888835', 'S3006888836', 'S3006888837', 'S3006888838', 'S3006888840', 'S3006888841', 'S3006888842', 'S3006888843', 'S3006888844', 'S3006888845', 'S3006888846', 'S3006888847', 'S3006888848', 'S3006888850', 'S3006888851', 'S3006888852', 'S3006888853', 'S3006888854', 'S3006888855', 'S3006888856', 'S3006888857', 'S3006888858', 'S3006888860', 'S3006888861', 'S3006888862', 'S3006888863', 'S3006888864', 'S3006888865', 'S3006888866', 'S3006888867', 'S3006888868', 'S3006888870', 'S3006888871', 'S3006888872', 'S3006888873', 'S3006888874', 'S3006888875', 'S3006888876', 'S3006888877', 'S3006888878', 'S3006888880', 'S3006888881', 'S3006888882', 'S3006888883', 'S3006888884', 'S3006888885', 'S3006888886', 'S3006888887', 'S3006888888', 'S3006888271', 'S3006888274', 'S3006888276', 'S3006888277', 'S3006888278', 'S3006888514', 'S3006888515', 'S3006888516', 'S3006888517', 'S3006888518', 'S3006888535', 'S3006888540', 'S3006888541', 'S3006888

In [9]:
def find_at_dggs_cell(dggs_cell):
    """
    Function for finding an array of Loci-i features by a DGGS AUxPIX Cell ID, eg "R6810000005"
    """
    dggs_prefix = 'http://ec2-52-63-73-113.ap-southeast-2.compute.amazonaws.com/AusPIX-DGGS-dataset/ausPIX/'

    index = dggs_cell.find(dggs_prefix)
    dggs_cell_id = dggs_cell
    if index==0:
        dggs_cell_id = dggs_cell[len(dggs_prefix):len(dggs_cell)]
    sql = f'select \
            \'http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/\'||sa1_main16 as asgs16_sa1, \
            \'http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/\'||sa2_main16 as asgs16_sa2, \
            \'http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel3/\'||sa3_code16 as asgs16_sa3, \
            \'http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/\'||lga_code19 as asgs16_lga, \
            \'http://linked.data.gov.au/dataset/asgs2016/statesuburb/\'||ssc_code16 as asgs16_ssc \
            FROM public."MainTable01" WHERE auspix_dggs=\'{dggs_cell_id}\''
#     print(sql)
    conn = psycopg2.connect(PG_ENDPOINT)
    db_cursor = conn.cursor()
    res = db_cursor.execute(sql)
    records = db_cursor.fetchall() 
    db_cursor.close()
    result = []
    for record in records:
        result.append(record)
    return result

In [10]:
%%time

loci_1 = find_at_dggs_cell('S3006887558')


CPU times: user 5.62 ms, sys: 1.18 ms, total: 6.8 ms
Wall time: 276 ms


In [11]:
%%time

loci_2 = find_at_dggs_cell('http://ec2-52-63-73-113.ap-southeast-2.compute.amazonaws.com/AusPIX-DGGS-dataset/ausPIX/S3006887558')


CPU times: user 4.82 ms, sys: 1.01 ms, total: 5.83 ms
Wall time: 218 ms


In [12]:
def gen_object_from_array(db_return):
    for item in db_return:
        print('{')
        print('  asgs16_sa1:', item[0])
        print('  asgs16_sa2:', item[1])
        print('  asgs16_sa3:', item[2])
        print('  asgs16_lga:', item[3])
        print('  asgs16_ssc:', item[4])
        print('}')

In [13]:
gen_object_from_array(loci_1)
gen_object_from_array(loci_2)

{
  asgs16_sa1: http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/60302107209
  asgs16_sa2: http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/603021072
  asgs16_sa3: http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel3/60302
  asgs16_lga: http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/63010
  asgs16_ssc: http://linked.data.gov.au/dataset/asgs2016/statesuburb/60214
}
{
  asgs16_sa1: http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/60302107209
  asgs16_sa2: http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/603021072
  asgs16_sa3: http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel3/60302
  asgs16_lga: http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/63010
  asgs16_ssc: http://linked.data.gov.au/dataset/asgs2016/statesuburb/60214
}
